In [2]:
pip install requests python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime
import time
import json

In [8]:
# load envrionment variables from the .env file
load_dotenv()

# get the API key from the .env file
api_key =os.getenv("API_KEY")

In [11]:
## function for getting lat & long encoding of cities

# Geocoding API endpoint
geocoding_url = "http://api.openweathermap.org/data/2.5/weather"

# List of cities
cities = ["Denver,CO,USA", "Austin,TX,USA", "Stuttgart,DE"]

# Loop through the cities and get their lat, lon
for city in cities:
    # Send GET request to the OpenWeatherMap Geocoding API
    response = requests.get(geocoding_url, params={
        'q': city,
        'appid': api_key
    })
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        lat = data['coord']['lat']
        lon = data['coord']['lon']
        print(f"City: {city} - Latitude: {lat}, Longitude: {lon}")
    else:
        print(f"Failed to get data for {city}")
        
##then store this information within the config file

City: Denver,CO,USA - Latitude: 39.7392, Longitude: -104.9847
City: Austin,TX,USA - Latitude: 30.2711, Longitude: -97.7437
City: Stuttgart,DE - Latitude: 48.7823, Longitude: 9.177


In [36]:
# api call to get the current weather 
import requests

def get_weather(api_key, lat, lon, exclude='minutely,daily,hourly', units='imperial', lang='en'):
    # Build the base URL for the OneCall API
    url = f"https://api.openweathermap.org/data/3.0/onecall"
    
    # Prepare the parameters for the API call
    params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key,
        'units': units,  # 'imperial' for Fahrenheit, 'metric' for Celsius
        'lang': lang      # Language for the response
    }
    
    # Add the 'exclude' parameter if it's provided
    if exclude:
        params['exclude'] = exclude
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Print or process the data
        return data
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [40]:
# Step 1: Load the config file
with open('cities_config.json', 'r') as f:
    config_data = json.load(f)
    
weather_data = []

# Step 2: Loop through each city and use the data for API requests
for city in config_data['cities']:
    latitude = city['latitude']
    longitude = city['longitude']
    
    weather_data.append(get_weather(api_key, latitude, longitude))

In [42]:
# File path within the "data" folder
file_path = 'data/raw_weather_data.json'

# Check if the file exists to decide whether to append or create new
if os.path.exists(file_path):
    # If the file exists, load the existing data, then append new data
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
        existing_data.extend(weather_data)
    
    # Append to the file
    with open(file_path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)
else:
    # If the file doesn't exist, create it and write the new data
    with open(file_path, 'w') as json_file:
        json.dump(weather_data, json_file, indent=4)

print(f"Data saved to {file_path}")

Data saved to data/raw_weather_data.json
